In [1]:
import pandas as pd
import numpy as np
from google.colab import drive

In [ ]:
drive.mount('/content/drive/')
!cp '/content/drive/MyDrive/TCC/VGG.zip' .
!unzip ./VGG.zip

In [3]:
###### DEFINES ######
DESCRIPTORS_PATH='./VGG/' 
SUMMARIZE_METHODS=['MEAN','MEDIAN','STD']

In [4]:
def get_segment_path(seg_id, seg_info):
  segment = seg_info[seg_info['seg_id'] == seg_id]
  return DESCRIPTORS_PATH + segment['video_name'].values[0] + '/' + segment['seg_id'].values[0] + '.csv'

def summarize_dataframe(df):
  metrics = []
  for method in SUMMARIZE_METHODS:
    if method == 'MEAN':
      mean = df.mean(axis=0)
      metrics.append(mean.to_numpy())
    if method == 'MEDIAN':
      median = df.median(axis=0)
      metrics.append(median.to_numpy())
    if method == 'STD':
      std = df.std(axis=0)
      metrics.append(std.to_numpy())
  return metrics

In [5]:
seg_info = pd.read_csv('https://raw.githubusercontent.com/newsdata/SoccerDB/master/dataset/video_dataset/seg_info.csv')
seg_info['seg_id'][0]

'73ed8378dc1811e897b86c96cfde8f_1_100_origin_000001_001629'

In [6]:
segments_metrics = [0]
df = pd.read_csv(get_segment_path('73ed8378dc1811e897b86c96cfde8f_1_100_origin_000001_001629', seg_info))
df = df.drop(df.columns[0], axis=1)
metrics = summarize_dataframe(df)
metrics = np.array(metrics).flatten()
segments_metrics[0] = metrics
len(segments_metrics[0])

1536

In [ ]:
segments_metrics = []
for idx, segment in enumerate(seg_info.to_numpy()):
  print(idx)
  df = pd.read_csv(get_segment_path(segment[0], seg_info))
  df = df.drop(df.columns[0], axis=1)
  metrics = summarize_dataframe(df)
  metrics = np.array(metrics).flatten()
  segments_metrics.append(metrics)

In [19]:
len(segments_metrics[0])

1536

In [20]:
columns = []
columns_per_metric = int(len(segments_metrics[0])/len(SUMMARIZE_METHODS))
for metric in SUMMARIZE_METHODS:
  metric_columns = [metric + '_' + str(i) for i in range(columns_per_metric)]
  columns.append(metric_columns)

In [ ]:
columns = list(np.array(columns).flatten())
columns

In [ ]:
metrics_df = pd.DataFrame(segments_metrics, columns=columns)
metrics_df['seg_id'] = seg_info['seg_id'].values
metrics_df['cls_id'] = seg_info['cls_id'].values

In [30]:
columns.insert(0, 'seg_id')
columns.insert(1, 'cls_id')
metrics_df = metrics_df[columns]

In [31]:
metrics_df

,seg_id,cls_id,MEAN_0,MEAN_1,MEAN_2,MEAN_3,MEAN_4,MEAN_5,MEAN_6,MEAN_7,...,STD_502,STD_503,STD_504,STD_505,STD_506,STD_507,STD_508,STD_509,STD_510,STD_511
0,73ed8378dc1811e897b86c96cfde8f_1_100_origin_00...,0,1.040726,2.908433,5.415557,13.534303,49.408671,7.962034,7.995172,28.459100,...,6.651066,12.199374,6.352187,8.361165,15.376564,15.860730,14.822319,0.672305,10.016976,32.897059
1,73ed8378dc1811e897b86c96cfde8f_1_100_origin_00...,0,18.146216,1.707686,4.817236,0.676112,19.250017,9.303244,5.053849,7.443971,...,9.899844,6.174066,4.070407,5.397249,10.849989,14.969843,8.271197,0.000000,5.937074,11.665777
2,73ed8378dc1811e897b86c96cfde8f_1_100_origin_00...,0,6.651225,9.502874,4.540128,2.841258,44.545239,9.295176,7.739557,36.735191,...,8.008388,8.146741,11.920758,4.556942,26.401784,22.236139,7.366693,2.371284,12.548949,20.013665
3,73ed8378dc1811e897b86c96cfde8f_1_100_origin_00...,0,2.883687,2.765757,8.453304,2.574604,44.726872,22.009967,9.790131,36.523389,...,3.463994,4.980632,8.714601,0.992832,18.120816,12.276475,9.022244,2.176398,8.175290,8.809985
4,73ed8378dc1811e897b86c96cfde8f_1_100_origin_00...,0,4.520419,5.779800,1.164119,1.504031,76.732398,17.580343,3.086707,34.092898,...,7.198753,11.361748,11.533055,0.911559,11.389960,18.005052,8.604114,2.448005,6.959429,18.423832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142570,ddce0e38d5ac11e8b2536c96cfde8f_100_origin_0000...,0,25.427629,1.323690,8.090103,0.900039,55.190589,32.435169,42.890131,26.021780,...,3.719114,12.297446,9.117653,11.437620,19.607629,14.154165,12.230365,0.000000,5.354950,17.929084
142571,ddce0e38d5ac11e8b2536c96cfde8f_100_origin_0000...,0,26.254412,1.252541,14.117269,3.748907,57.738189,33.056517,17.373277,19.037986,...,5.053526,12.297322,8.222824,9.776666,19.345532,14.360757,8.234063,0.378767,6.694472,39.687998
142572,ddce0e38d5ac11e8b2536c96cfde8f_100_origin_0000...,0,53.289957,1.565482,9.580962,0.000000,62.946877,22.718227,33.920024,12.351120,...,1.995447,9.030263,4.691851,7.284102,18.587420,8.953388,3.409957,0.000000,2.637709,2.421959
142573,ddce0e38d5ac11e8b2536c96cfde8f_115-8a0b0941-dd...,9,25.521953,1.378544,16.250499,0.000000,82.817625,52.935982,16.828717,5.987653,...,6.837419,21.049185,5.778025,9.117961,15.157445,5.043188,5.732983,7.560027,4.915330,9.009602


In [32]:
metrics_df.to_csv('./mean_median_std_segments.csv')

In [33]:
!ls -sh

total 16G
4.0K drive			   4.0K sample_data   13G VGG.zip
3.0G mean_median_std_segments.csv   36K VGG


In [35]:
!mv './mean_median_std_segments.csv' '/content/drive/MyDrive/TCC/'